<a href="https://colab.research.google.com/github/gopinathak-geek/novozymes-enzyme-stability-prediction/blob/main/NovoModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/train.csv"
train_updates_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/train_updates_20220929.csv"
test_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/test.csv"

train_df = pd.read_csv(train_csv)
train_updates_df = pd.read_csv(train_updates_csv)
test_df = pd.read_csv(test_csv)

In [3]:
seq_ids_to_delete = train_updates_df[train_updates_df["pH"].isnull()]["seq_id"].values
seq_ids_to_replace = train_updates_df[train_updates_df["pH"].notnull()]["seq_id"].values

In [4]:
train_df.drop(train_df[train_df.seq_id.isin(seq_ids_to_delete)].index, inplace=True)

In [5]:
train_df.loc[train_df.seq_id.isin(seq_ids_to_replace), ["pH", "tm"]] = train_updates_df[train_updates_df.seq_id.isin(seq_ids_to_replace)][["pH", "tm"]].values

In [6]:
null_seq_ids = train_df[train_df["pH"].isnull()]["seq_id"].values

In [7]:
train_df.drop(train_df[train_df.seq_id.isin(null_seq_ids)].index, inplace=True)

In [8]:
train_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28695 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            28695 non-null  int64  
 1   protein_sequence  28695 non-null  object 
 2   pH                28695 non-null  float64
 3   data_source       27727 non-null  object 
 4   tm                28695 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.3+ MB


In [9]:
train_df.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
seq_id,28695.0,15631.432166,9235.099078,0.00,7447.5,15443.0,23758.5,31389.0
pH,28695.0,6.872467,0.793184,1.99,7.0,7.0,7.0,11.0
tm,28695.0,51.385604,12.076609,25.10,43.7,48.8,54.6,130.0


In [10]:
import sys
print (sys.version)


3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]


In [11]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy import stats
import torchvision.transforms as T


In [12]:
def getMolecularWeight(aminoacid):
  if aminoacid == "A": return 89
  elif aminoacid == "R": return 174
  elif aminoacid == "N": return 132
  elif aminoacid == "D": return 133
  elif aminoacid == "B": return 133
  elif aminoacid == "C": return 121
  elif aminoacid == "Q": return 146
  elif aminoacid == "E": return 147
  elif aminoacid == "Z": return 147
  elif aminoacid == "G": return 75
  elif aminoacid == "H": return 155
  elif aminoacid == "I": return 131
  elif aminoacid == "L": return 131
  elif aminoacid == "K": return 146
  elif aminoacid == "M": return 149
  elif aminoacid == "F": return 165
  elif aminoacid == "P": return 115
  elif aminoacid == "S": return 105
  elif aminoacid == "T": return 119
  elif aminoacid == "W": return 204
  elif aminoacid == "Y": return 181
  elif aminoacid == "V": return 117
  else: return 0

In [13]:
def proteinSequenceToAmioAcidMolecularWeightWithPh(data):
  amionAcidSequence = []
  img = []
  label = []
  for index, row in data.iterrows():
    ph = row['pH']
    tm = row['tm']
    molecularWeightOfAminoAcidInSequence = []
    pHofProteinSequence = []
    for aminoacid in (row['protein_sequence']):
      molecularWeight = getMolecularWeight(aminoacid)
      molecularWeightOfAminoAcidInSequence.append(molecularWeight/110)
      pHofProteinSequence.append(ph)
    #a = torch.FloatTensor(np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence]))
    #a = a[None, :]
    #a = preprocess(np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence]))
    a = np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence])
    a = a[None, :]
    img.append(a)
    label.append(tm)
    #amionAcidSequence.append((a, tm))
    #break
  return [img, label]

In [14]:
img, label = proteinSequenceToAmioAcidMolecularWeightWithPh(train_df)

In [15]:
import torchvision.transforms as T
from PIL import Image

preprocess = T.Compose([
   T.Resize((2,10)),
])

def change_shape(data):
    torch_img = []
    for im in (data):
        im = torch.FloatTensor(im)
        #transform = T.ToPILImage()
        #img = transform(im)
        x = preprocess(im)
        x = x.cpu().detach().numpy()
        torch_img.append(x)
    return torch_img

In [16]:
img = change_shape(img)

In [17]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [18]:
train_data = TrainData(torch.FloatTensor(img), torch.FloatTensor(label))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [19]:
BATCH_SIZE = 128
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [20]:
class NovoNet(nn.Module):
    def __init__(self, name=None):
        super(NovoNet, self).__init__()
        if name:
            self.name = name
        self.conv1 = nn.Conv2d(1, 10, 2) 
        #self.pool = nn.MaxPool2d((3, 2), stride=(2, 2))
        self.fc1 = nn.Linear(10 * 9 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, 9 * 1 * 10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [22]:
net = NovoNet(name='Novonet')
net.to(device)

Novonet: total params: 21219


NovoNet(
  (conv1): Conv2d(1, 10, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=90, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

In [23]:
LEARNING_RATE = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [25]:
EPOCHS = 50
for epoch in range(EPOCHS):
    net.train()
    score = []
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        label = label.unsqueeze(1)
    
        #==========Forward pass===============
        preds = net(img)
        loss = criterion(preds, label)
        #==========backward pass==============

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        a = stats.spearmanr(preds.detach().cpu().numpy(), label.cpu().numpy())
        score.append(a.correlation)
    print(f'EPOCH score {epoch+1}: {np.mean(score)}')

EPOCH score 1: -0.006740705119932366
EPOCH score 2: -0.004746099348018418
EPOCH score 3: -0.0032117921389270437
EPOCH score 4: 0.00278792796898146
EPOCH score 5: 0.010262915896133594
EPOCH score 6: 0.017350019560169686
EPOCH score 7: 0.019670353177218113
EPOCH score 8: 0.02656837279040166
EPOCH score 9: 0.028916767049756657
EPOCH score 10: 0.03834582297978721
EPOCH score 11: 0.02849126904400921
EPOCH score 12: 0.04012897302448542
EPOCH score 13: 0.040158672146822595
EPOCH score 14: 0.04614890139092245
EPOCH score 15: 0.05169938365611503
EPOCH score 16: 0.04994604428102583
EPOCH score 17: 0.054001227542843346
EPOCH score 18: 0.05697194133792863
EPOCH score 19: 0.05808885403009493
EPOCH score 20: 0.05913737204208473
EPOCH score 21: 0.060895382255598304
EPOCH score 22: 0.06476353311997106
EPOCH score 23: 0.06358441033241967
EPOCH score 24: 0.06288966262435965
EPOCH score 25: 0.06489588911991771
EPOCH score 26: 0.07085516027737593
EPOCH score 27: 0.06738408229710134
EPOCH score 28: 0.06915